<a href="https://colab.research.google.com/github/siddhartha201/CSC-752-Project/blob/main/Late_fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt


In [5]:
!git clone https://github.com/siddhartha201/CSC-752-Project

fatal: destination path 'CSC-752-Project' already exists and is not an empty directory.


In [6]:
dir = '/content/CSC-752-Project'
# Load your pre-trained models
cnn_model = load_model(dir +'/CNN_model.h5')
nlp_model = load_model(dir +'/NLP_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
text_data = dir + "//BrEaST-Lesions-USG-clinical-data-Dec-15-2023.xlsx"

In [10]:
df = pd.read_excel(text_data)

In [13]:
# Preprocessing
columns_to_drop = ["Image_filename", "Mask_tumor_filename", "Mask_other_filename", "Pixel_size", "Verification", "BIRADS"]
new_df = df.drop(columns=columns_to_drop)

# Handling missing Age
new_df_filtered = new_df[new_df['Age'] != 'not available']
new_df_filtered.loc[:, 'Age'] = pd.to_numeric(new_df_filtered['Age'])
age_mean_corrected = new_df_filtered['Age'].mean()

# Convert 'Age' to float first, replace 'not available' with the mean age, and then cast to int
new_df['Age'] = pd.to_numeric(new_df['Age'], errors='coerce')  # Convert all 'Age' values to numeric, turning errors into NaNs
new_df['Age'] = new_df['Age'].fillna(age_mean_corrected)  # Fill NaN values (previous 'not available' entries) with the mean
new_df['Age'] = new_df['Age'].astype(int)  # Now cast to integer

# Drop rows where the 'Classification' column does not contain 'benign' or 'malignant'
final_df = new_df[(new_df['Classification'] == 'benign') | (new_df['Classification'] == 'malignant')]

# Separating labels and features
label = final_df['Classification']
final_df = final_df.drop('Classification', axis=1)


In [14]:
# NLP feature extraction

non_numerical_columns = [col for col in final_df.columns if final_df[col].dtype == 'object' and col not in ['CaseID', 'Age']]
# TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=100, stop_words='english')

for column in non_numerical_columns:
    # Convert column to string
    column_as_string = final_df[column].astype(str).fillna('')
    tfidf_matrix = tfidf_vectorizer.fit_transform(column_as_string)

    feature_names = [f"{column}_{word}" for word in tfidf_vectorizer.get_feature_names_out()]
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

    # Concatenate TF-IDF
    if 'final_numerical_df' in locals():
        final_numerical_df = pd.concat([final_numerical_df, tfidf_df], axis=1)
    else:
        final_numerical_df = tfidf_df.copy()


final_numerical_df = pd.concat([final_df[['CaseID', 'Age']].reset_index(drop=True), final_numerical_df.reset_index(drop=True)], axis=1)

In [15]:
# split the data
X = final_numerical_df

encoder = LabelEncoder()
y = encoder.fit_transform(label)

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((201, 156), (51, 156), (201,), (51,))

In [16]:
text_predictions = nlp_model.predict(X_test)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_116" is incompatible with the layer: expected axis -1 of input shape to have value 2809, but received input with shape (32, 156)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 156), dtype=float32)
  • training=False
  • mask=None